# Modelos probabilisticos - Naive Bayes

* Em determinados problemas, é possível fazer uma relação entre variáveis por meio de probabilidades
* O naive bayes é um modelo voltado para esta visão, tendo como base o teorema de Bayes
* Suposição de independência ou não correlação entre as variáveis do conjunto de dados sendo utilizadas para resolver um determinado problema
* Variáveis igualmente importantes
* Formulação de acordo com o teorema de Bayes
  * Utilizado para cálculo da probabilidade de um evento A dado que um evento B ocorreu
  * P(B|A): probabilidade de B acontecer dado que A ocorreu
  * P(A): probabilidade de A ocorrer
  * P(B): probabilidade de B ocorrer
![Title](../imgs/teorema_bayes.webp)

# Como assim professora?
![Title](../imgs/fine.jpg)

# Exemplo
![Title](../imgs/tabela-bayes.png)

𝑃(𝑆𝑖𝑚|𝐸𝑛𝑠𝑜𝑙𝑎𝑟𝑎𝑑𝑜)=(𝑃(𝐸𝑛𝑠𝑜𝑙𝑎𝑟𝑎𝑑𝑜|𝑆𝑖𝑚)∗𝑃(𝑆𝑖𝑚))/𝑃(𝐸𝑛𝑠𝑜𝑙𝑎𝑟𝑎𝑑𝑜)


𝑃(𝑆𝑖𝑚)=9/14=0,64

𝑃(𝐸𝑛𝑠𝑜𝑙𝑎𝑟𝑎𝑑𝑜)=5/14=0,36

𝑃(𝐸𝑛𝑠𝑜𝑙𝑎𝑟𝑎𝑑𝑜|𝑆𝑖𝑚)=3/9=0,33

P(Sim | Ensolarado) = 0,33 * 0,64 / 0,36 = 0,60

# Uso em NLP

* O algoritmo de Bayes é uma das técnicas simples mais utilizadas para NLP em aplicações de classificação
* Palavras com maior probabilidade de ocorrer em uma determinada classe determinam um padrão

![Title](../imgs/bayes_nlp.png)

## Variações do Naive Bayes

Algoritmos de ML dependem principalmente da natureza dos seus dados! Sendo assim, naturalmente, diferentes formas de organizar a base de dados retornam diferentes resultados para o algoritmo de Bayes. Dessa forma, temos variações:

* Gaussian Naive Bayes: Utilizado na classificação assumindo uma distribuição normal.
* Bernoulli Naive Bayes: O modelo binomial é útil se os vetores são binários (0 ou 1)
* Multinomial Naive Bayes: Esse algoritmo usa os dados em uma distribuição multinomial, que é uma generalização da distribuição binomial. Se, por exemplo, considerarmos um problema de classificação de texto. Podemos considerar tentativas de Bernoulli, onde ao invés de verificarmos a palavra que ocorre no documento, verificamos a contagem de vezes que a palavra apareceu no texto.

In [39]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.stem.porter import *
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB

# Bibliotecas para o modelo
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB

In [2]:
nltk.download('stopwords')
nltk.download('punkt')

# Define as stopwords em inglês
sw_english = set(stopwords.words('english'))

# Instância o PorterStemmer
stemmer = PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cecilia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/cecilia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
df = pd.read_csv('../datasets/spamraw.csv')
df

,type,text
0,ham,Hope you are having a good week. Just checking in
1,ham,K..give back my thanks.
2,ham,Am also doing in cbe only. But have to pay.
3,spam,"complimentary 4 STAR Ibiza Holiday or £10,000 ..."
4,spam,okmail: Dear Dave this is your final notice to...
...,...,...
5554,ham,You are a great role model. You are giving so ...
5555,ham,"Awesome, I remember the last time we got someb..."
5556,spam,"If you don't, your prize will go to another cu..."
5557,spam,"SMS. ac JSco: Energy is high, but u may not kn..."


In [20]:
def preprocessing(string):
    ###
    # tokenização
    words = word_tokenize(string)
    ###
    # Remove Stopwords
    filtered_words = []
    for w in words:
        if w not in sw_english:
            filtered_words.append(w)
    ###
    # Aplica o Stemming
    stem_words = []
    for w in filtered_words:
        s_words = stemmer.stem(w)
        stem_words.append(s_words)
    ###
    # Retorna a lista de palavras pré-processadas
    return stem_words

In [21]:
df["filtered_words"] = df['text'].apply(lambda x: preprocessing(x))
df['join_words'] = df['filtered_words'].apply(lambda x: ' '.join(x))

In [35]:
X = df['join_words']
y = df['type'].replace({'ham':0, 'spam': 1})

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                     y,
                                                     test_size = 0.3,
                                                     random_state = 42)

In [37]:
tfidf = TfidfVectorizer(use_idf = True)

X_train_tfidf = tfidf.fit_transform(X_train).todense()
X_test_tfidf  = tfidf.transform(X_test).todense()

In [40]:
# Instanciando o modelo
model = GaussianNB()

# treinamento do modelo
model.fit(X_train_tfidf, y_train)

# Gerando as predições
y_pred = model.predict(X_test_tfidf)

# Print do classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.89      0.93      1457
           1       0.53      0.88      0.66       211

    accuracy                           0.88      1668
   macro avg       0.75      0.88      0.79      1668
weighted avg       0.92      0.88      0.90      1668



In [42]:
# Instanciando o modelo
model = MultinomialNB()

# treinamento do modelo
model.fit(X_train_tfidf, y_train)

# Gerando as predições
y_pred = model.predict(X_test_tfidf)

# Print do classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      1457
           1       1.00      0.73      0.85       211

    accuracy                           0.97      1668
   macro avg       0.98      0.87      0.91      1668
weighted avg       0.97      0.97      0.96      1668



In [45]:
# Carrega a função CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# instancia o transformador
cv = CountVectorizer()

# Transforma os dados nas matrizes de saída
X_train_cv = cv.fit_transform(X_train).toarray()
X_test_cv = cv.transform(X_test).toarray()

In [33]:
# Instanciando o modelo
model = GaussianNB()

# treinamento do modelo
model.fit(X_train_cv, y_train)

# Gerando as predições
y_pred = model.predict(X_test_cv)

# Print do classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.90      0.94      1457
           1       0.58      0.92      0.71       211

    accuracy                           0.90      1668
   macro avg       0.78      0.91      0.83      1668
weighted avg       0.94      0.90      0.91      1668



### MultinomialNB

Como vimos que o Multinomial Naive Bayes é indicado quando nossos dados estão em uma estrutura baseada em contagens, podemos utiliza-lo para o nosso problema e comparar os resultados obtidos anteriormente

In [44]:
# Instanciando o modelo
model = MultinomialNB()

# treinamento do modelo
model.fit(X_train_tfidf, y_train)

# Gerando as predições
y_pred = model.predict(X_test_tfidf)

# Print do classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      1457
           1       1.00      0.73      0.85       211

    accuracy                           0.97      1668
   macro avg       0.98      0.87      0.91      1668
weighted avg       0.97      0.97      0.96      1668



In [46]:
# Instanciando o modelo
model = MultinomialNB()

# treinamento do modelo
model.fit(X_train_cv, y_train)

# Gerando as predições
y_pred = model.predict(X_test_cv)

# Print do classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1457
           1       0.96      0.90      0.93       211

    accuracy                           0.98      1668
   macro avg       0.97      0.95      0.96      1668
weighted avg       0.98      0.98      0.98      1668



### Outros algoritmos de ML

In [48]:
# Instanciando o modelo
model = RandomForestClassifier()

# treinamento do modelo
model.fit(X_train_tfidf, y_train)

# Gerando as predições
y_pred = model.predict(X_test_tfidf)

# Print do classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      1.00      0.99      1457
           1       0.99      0.82      0.89       211

    accuracy                           0.98      1668
   macro avg       0.98      0.91      0.94      1668
weighted avg       0.98      0.98      0.97      1668

